In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

import utils.read_data as rd
import utils.preprocessing as pre
import utils.join_race_data as jrd

In [2]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path

In [3]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH")
DATA_PATH = GOOGLE_DRIVE_PATH + '/scraping/'

In [4]:
TARGET = '20201128'

In [5]:
df_target_horse = rd.read_horse_csv(DATA_PATH + TARGET + '/')
df_target_race = rd.read_race_csv(DATA_PATH + TARGET + '/')

/home/websci/.pyenv/versions/3.7.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/websci/.pyenv/versions/3.7.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/websci/.pyenv/versions/3.7.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (0,16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
df_target_horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639818 entries, 0 to 639817
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   race_id             639818 non-null  float64
 1   rank                639818 non-null  object 
 2   frame_number        639818 non-null  int8   
 3   horse_number        639818 non-null  int8   
 4   horse_id            639818 non-null  int64  
 5   sex_and_age         639818 non-null  object 
 6   burden_weight       639818 non-null  float16
 7   rider_id            639818 non-null  int32  
 8   goal_time           634285 non-null  object 
 9   goal_time_dif       589557 non-null  object 
 10  half_way_rank       636449 non-null  object 
 11  last_time           634280 non-null  float16
 12  odds                639818 non-null  object 
 13  popular             637487 non-null  float16
 14  horse_weight        639818 non-null  object 
 15  tamer_id            639818 non-nul

In [7]:
arr_dir = []
for f in os.listdir(DATA_PATH):
  if os.path.isdir(os.path.join(DATA_PATH, f)):
    arr_dir.append(f)
    
print(arr_dir)


['20201128']


In [8]:
df_horse = pd.DataFrame()
df_race = pd.DataFrame()
for dir in arr_dir:
    df_horse = df_horse.append(rd.read_horse_csv(DATA_PATH + dir + '/'))
    df_race = df_race.append(rd.read_race_csv(DATA_PATH + dir + '/'))

In [9]:
df_horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639818 entries, 0 to 639817
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   race_id             639818 non-null  float64
 1   rank                639818 non-null  object 
 2   frame_number        639818 non-null  int8   
 3   horse_number        639818 non-null  int8   
 4   horse_id            639818 non-null  int64  
 5   sex_and_age         639818 non-null  object 
 6   burden_weight       639818 non-null  float16
 7   rider_id            639818 non-null  int32  
 8   goal_time           634285 non-null  object 
 9   goal_time_dif       589557 non-null  object 
 10  half_way_rank       636449 non-null  object 
 11  last_time           634280 non-null  float16
 12  odds                639818 non-null  object 
 13  popular             637487 non-null  float16
 14  horse_weight        639818 non-null  object 
 15  tamer_id            639818 non-nul

In [10]:
df_race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44584 entries, 0 to 44583
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   race_id              44584 non-null  float64
 1   race_round           44584 non-null  object 
 2   race_title           44584 non-null  object 
 3   race_course          44584 non-null  object 
 4   weather              44584 non-null  object 
 5   ground_status        44584 non-null  object 
 6   time                 44584 non-null  object 
 7   date                 44584 non-null  object 
 8   where_racecourse     44584 non-null  object 
 9   race_class           44584 non-null  object 
 10  running_condition    42838 non-null  object 
 11  total_horse_number   44584 non-null  int8   
 12  frame_number_first   44584 non-null  int8   
 13  horse_number_first   44584 non-null  int8   
 14  frame_number_second  44584 non-null  int8   
 15  horse_number_second  44584 non-null 

In [11]:
df = pd.merge(df_horse, df_race, on='race_id', how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 639818 entries, 0 to 639817
Data columns (total 48 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   race_id               639818 non-null  float64
 1   rank                  639818 non-null  object 
 2   frame_number          639818 non-null  int8   
 3   horse_number          639818 non-null  int8   
 4   horse_id              639818 non-null  int64  
 5   sex_and_age           639818 non-null  object 
 6   burden_weight         639818 non-null  float16
 7   rider_id              639818 non-null  int32  
 8   goal_time             634285 non-null  object 
 9   goal_time_dif         589557 non-null  object 
 10  half_way_rank         636449 non-null  object 
 11  last_time             634280 non-null  float16
 12  odds                  639818 non-null  object 
 13  popular               637487 non-null  float16
 14  horse_weight          639818 non-null  object 
 15  

In [12]:
df.head()

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,...,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3
0,2.013050e+11,1,5,10,2010103744,牝3,54.0,422,1:20.9,NaN,...,150,120,660,700,290,180,340,"1,380",730,"3,720"
1,2.013050e+11,2,2,3,2010104843,牝3,54.0,1043,1:21.2,1.3/4,...,150,120,660,700,290,180,340,"1,380",730,"3,720"
2,2.013050e+11,3,6,11,2010105413,牡3,56.0,660,1:21.3,1/2,...,150,120,660,700,290,180,340,"1,380",730,"3,720"
3,2.013050e+11,4,4,8,2010103463,牝3,54.0,1122,1:21.3,アタマ,...,150,120,660,700,290,180,340,"1,380",730,"3,720"
4,2.013050e+11,5,8,15,2010103480,牡3,56.0,684,1:21.4,3/4,...,150,120,660,700,290,180,340,"1,380",730,"3,720"


In [13]:
len(df_horse)

639818

In [14]:
arr_horse_id = list(set(df_target_horse['horse_id']))
# mini size
# arr_horse_id = arr_horse_id[:1000]
len(arr_horse_id)

1000

In [15]:
arr_horse_df = []
arr_horse_race_id = []
for horse_id in tqdm(arr_horse_id):
    arr_horse_df.append(df.query('horse_id == @horse_id'))
    arr_horse_race_id.append(list(df_target_horse.query('horse_id == @horse_id').race_id))

100%|██████████| 1000/1000 [00:05<00:00, 173.59it/s]


In [16]:
arr_res = Parallel(n_jobs=-1, verbose=5)([delayed(jrd.join_n_race_for_train_data)(df, arr_race_id, n_race=5)
                                          for df, arr_race_id in zip(arr_horse_df, arr_horse_race_id)])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   49.7s


KeyboardInterrupt: 

In [ ]:
result = pd.concat((df.T for df in tqdm(arr_res)), axis=1).T

#result = pd.DataFrame()
#for df in tqdm(arr_res):
#    result = pd.concat([result, df])

In [ ]:
result.reset_index(inplace=True, drop=True)

In [ ]:
result.info()

In [ ]:
result.columns[30:60]

In [ ]:
result.sort_values('race_id', ascending=False)

In [ ]:
result.to_csv(GOOGLE_DRIVE_PATH + '/train_data/horse_race_' + TARGET + '.csv')

# テスト

In [ ]:
len(df_horse)

In [ ]:
df_horse.groupby('horse_id')['race_id'].count().max()

In [ ]:
df_horse_min = df_horse.iloc[0:10000:10, :]

In [ ]:
df_horse_min

In [ ]:
def add_n_race_data(df_race, df_horse, n_race=1):
    res = pd.DataFrame()
    horse_ids = list(set(df_horse['horse_id']))
    df = pd.merge(df_horse, df_race, on='race_id', how='left')
    for horse_id in tqdm(horse_ids):
        df_horse_tmp = df.query('horse_id == @horse_id')
        df_horse_tmp = df_horse_tmp.sort_values("race_id")
        race_ids = list(set(df_horse_tmp['race_id']))
        for race_id in race_ids:
            df_race_tmp = df_horse_tmp.query('race_id == @race_id').reset_index(drop=True)
            df_horse_low_race_id = df_horse_tmp.query('race_id <= @race_id')
            columns = df_horse_low_race_id.columns
            for n in range(1, n_race+1):
                add_columns = list(map(lambda c: c+f"_{n}", columns))
                add_race = df_horse_low_race_id.iloc[-1-n:-n, :].reset_index(drop=True)
                add_race.columns = add_columns
                df_race_tmp = pd.concat([df_race_tmp, add_race], axis=1)
                    
            res = pd.concat([res, df_race_tmp])

    return res

In [ ]:
result = add_n_race_data(df_race, df_horse_min, n_race=15)
#result = add_n_race_data(df_race, df_horse_min, n_race=5)

In [ ]:
result

In [ ]:
result.to_csv(GOOGLE_DRIVE_PATH + '/data/horse_race.csv')

In [ ]:
result.info()

In [ ]:
for column in result.columns:
    print(column)

In [ ]:
def add_n_race_data_para(df):
    res = pd.DataFrame()
    n_race = 15
    df = df.sort_values("race_id")
    race_ids = list(set(df['race_id']))
    for race_id in race_ids:
        df_race_tmp = df.query('race_id == @race_id').reset_index(drop=True)
        df_horse_low_race_id = df.query('race_id <= @race_id')
        columns = df_horse_low_race_id.columns
        for n in range(1, n_race+1):
            add_columns = list(map(lambda c: c+f"_{n}", columns))
            add_race = df_horse_low_race_id.iloc[-1-n:-n, :].reset_index(drop=True)
            add_race.columns = add_columns
            df_race_tmp = pd.concat([df_race_tmp, add_race], axis=1)

        res = pd.concat([res, df_race_tmp])

    return res

In [ ]:
def create_horse_data(n_race, df_race, df_horse):
    df_race = df_race.reset_index(drop=True)
    race_id = df_race.iloc[0, 0]
    df_horse.sort_values("race_id", inplace=True)
    df_hourse = df_horse.query('race_id < @race_id')
    columns = df_horse.columns
    for n in range(1, n_race+1):
        add_columns = list(map(lambda c: c+f"_{n}", columns))
        add_race = df_horse.iloc[-1-n:-n, :].reset_index(drop=True)
        add_race.columns = add_columns
        df_race = pd.concat([df_race, add_race], axis=1)
    return df_race

In [ ]:
def extract_previous_race(n, df):
    return df.loc[:, f"race_year_{n}": f"horse_ranking_index_{n}"]

In [ ]:
df_horse_test = df_horse.query('horse_id == 2010103480')
df_horse_test

In [ ]:
# 3レース前までのデータを列に追加
result = create_horse_data(3, df_race_test, df_horse_test)
result

In [ ]:
extract_previous_race(1, result)

# 古いコード

In [ ]:
df = pd.read_csv("data/data_2400.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df["horse_name"].value_counts()

## 指定した馬のnレース前までのデータを列に追加する

In [ ]:
test_df = df[df["horse_name"] == "ステイブラビッシモ"].loc[:, :"horse_ranking_index"]
test_df

## 日時情報を整形
日時情報を使ってデータフレームをソートしたデータから、nレース前までのデータを取得するために行う。

In [ ]:
def add_date_data(df, year_column_name, month_column_name,  day_column_name):
    df["date"] = df.apply(lambda row: f"{row[year_column_name]}-{row[month_column_name]}-{row[day_column_name]}", axis=1)
    df["date"] = pd.to_datetime(df["date"])

In [ ]:
add_date_data(test_df, "race_year", "race_month", "race_day")
test_df.info()

## nレース前までのデータを列に追加する

In [ ]:
def create_horse_data(n_race, df):
    df.sort_values("date", inplace=True)
    race_df = df.iloc[-1:, :].reset_index(drop=True)
    columns = race_df.columns
    for n in range(1, n_race+1):
        add_columns = list(map(lambda c: c+f"_{n}", columns))
        add_race = df.iloc[-1-n:-n, :].reset_index(drop=True)
        add_race.columns = add_columns
        race_df = pd.concat([race_df, add_race], axis=1)
    return race_df

In [ ]:
# 3レース前までのデータを列に追加
result = create_horse_data(3, test_df)
result

In [ ]:
def extract_previous_race(n, df):
    return df.loc[:, f"race_year_{n}": f"horse_ranking_index_{n}"]

In [ ]:
extract_previous_race(1, result)

In [ ]:
extract_previous_race(2, result)

In [ ]:
extract_previous_race(3, result)